<a href="https://colab.research.google.com/github/biamuniz/guia-ne/blob/master/C%C3%B3pia_de_ova_teste.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Para visualizar só um arquivo

In [ ]:

import xml.etree.ElementTree as ET
import pandas as pd

# ====== 1. Carregar o XML ======
xml_file = "/content/2220995762562124.xml"  # coloque o caminho do seu arquivo
tree = ET.parse(xml_file)
root = tree.getroot()

# ====== 2. Dados Gerais ======
dados_gerais = root.find("DADOS-GERAIS")
tabela_dados_gerais = pd.DataFrame([{
    "id_curriculo": root.attrib.get("NUMERO-IDENTIFICADOR"),
    "nome_completo": dados_gerais.attrib.get("NOME-COMPLETO"),
    "nome_citacao": dados_gerais.attrib.get("NOME-EM-CITACOES-BIBLIOGRAFICAS"),
    "nacionalidade": dados_gerais.attrib.get("PAIS-DE-NACIONALIDADE"),
    "cidade_nascimento": dados_gerais.attrib.get("CIDADE-NASCIMENTO"),
    "uf_nascimento": dados_gerais.attrib.get("UF-NASCIMENTO"),
    "pais_nascimento": dados_gerais.attrib.get("PAIS-DE-NASCIMENTO")
}])
# ====== 3. Formação Acadêmica ======
formacoes = []
for f in dados_gerais.find("FORMACAO-ACADEMICA-TITULACAO"):
    formacoes.append({
        "id_curriculo": root.attrib.get("NUMERO-IDENTIFICADOR"),
        "tipo": f.tag,
        "instituicao": f.attrib.get("NOME-INSTITUICAO"),
        "curso": f.attrib.get("NOME-CURSO"),
        "ano_inicio": f.attrib.get("ANO-DE-INICIO"),
        "ano_conclusao": f.attrib.get("ANO-DE-CONCLUSAO"),
        "status": f.attrib.get("STATUS-DO-CURSO"),
        "titulo_trabalho": f.attrib.get("TITULO-DO-TRABALHO-DE-CONCLUSAO-DE-CURSO", "")
    })
tabela_formacao = pd.DataFrame(formacoes)

# ====== 4. Idiomas ======
idiomas = []
for i in dados_gerais.find("IDIOMAS"):
    idiomas.append({
        "id_curriculo": root.attrib.get("NUMERO-IDENTIFICADOR"),
        "idioma": i.attrib.get("DESCRICAO-DO-IDIOMA"),
        "leitura": i.attrib.get("PROFICIENCIA-DE-LEITURA"),
        "escrita": i.attrib.get("PROFICIENCIA-DE-ESCRITA"),
        "fala": i.attrib.get("PROFICIENCIA-DE-FALA"),
        "compreensao": i.attrib.get("PROFICIENCIA-DE-COMPREENSAO"),
    })
tabela_idiomas = pd.DataFrame(idiomas)

# ====== 5. Produção Técnica (Apresentações) ======
producoes = []
for apresentacao in root.findall(".//APRESENTACAO-DE-TRABALHO"):
    dados = apresentacao.find("DADOS-BASICOS-DA-APRESENTACAO-DE-TRABALHO")
    detalhe = apresentacao.find("DETALHAMENTO-DA-APRESENTACAO-DE-TRABALHO")
    producoes.append({
        "id_curriculo": root.attrib.get("NUMERO-IDENTIFICADOR"),
        "titulo": dados.attrib.get("TITULO"),
        "natureza": dados.attrib.get("NATUREZA"),
        "ano": dados.attrib.get("ANO"),
        "evento": detalhe.attrib.get("NOME-DO-EVENTO"),
        "cidade": detalhe.attrib.get("CIDADE-DA-APRESENTACAO"),
        "instituicao": detalhe.attrib.get("INSTITUICAO-PROMOTORA")
    })
tabela_producoes = pd.DataFrame(producoes)

atuacoes = []
for atuacao in root.findall(".//ATUACAO-PROFISSIONAL"):
  instituicao = atuacao.attrib.get("NOME-INSTITUICAO", "")
  cod_instituicao = atuacao.attrib.get("CODIGO-INSTITUICAO", "")
  for vinculo in atuacao.findall("VINCULOS"):
    atuacoes.append({
        "Instituição": instituicao,
        "Código Instituição": cod_instituicao,
        "Tipo Vínculo": vinculo.attrib.get("TIPO-DE-VINCULO", ""),
        "Enquadramento": vinculo.attrib.get("ENQUADRAMENTO-FUNCIONAL", ""),
        "Outro Enquadramento": vinculo.attrib.get("OUTRO-ENQUADRAMENTO-FUNCIONAL-INFORMADO", ""),
        "Início": f"{vinculo.attrib.get('MES-INICIO','')}/{vinculo.attrib.get('ANO-INICIO','')}",
        "Fim": f"{vinculo.attrib.get('MES-FIM','')}/{vinculo.attrib.get('ANO-FIM','')}",
        "Carga Horária Semanal": vinculo.attrib.get("CARGA-HORARIA-SEMANAL", ""),
        "Dedicacao Exclusiva": vinculo.attrib.get("FLAG-DEDICACAO-EXCLUSIVA", ""),
        "Vínculo Empregatício": vinculo.attrib.get("FLAG-VINCULO-EMPREGATICIO", ""),
        "Outras Informações": vinculo.attrib.get("OUTRAS-INFORMACOES", "")
        })

tabela_atuacoes = pd.DataFrame(atuacoes)

In [ ]:
# ====== 6. Mostrar os DataFrames ======
print("\n📌 Dados Gerais:")
tabela_dados_gerais

In [ ]:
print("\n📌 Formação Acadêmica:")
tabela_formacao

In [ ]:
"\n📌 Idiomas:"
tabela_idiomas

In [ ]:
print("\n📌 Produções Técnicas:")
tabela_producoes

In [ ]:
print("\n📌 Atuação:")
tabela_atuacoes

## Processamento em lote
O que esse código faz:


*   Procura todos os .xml dentro de xml_files (e subpastas).
*   Filtra aqueles que têm em formação ou atuação uma das universidades da lista.
*   Adiciona as informações às tabelas.
*   Copia o XML original para a pasta xml_files/FILTRADO/.
*   Salva também os CSVs com os dados filtrados.

In [ ]:
import xml.etree.ElementTree as ET
import pandas as pd
import glob
import os
import shutil

# ==============================
# Lista de universidades e institutos para filtro
# ==============================
instituicoes = [
    # --- Universidades Federais ---
    "Universidade Federal da Bahia",
    "Universidade Federal do Sul da Bahia",
    "Universidade Federal do Recôncavo da Bahia",
    "Universidade Federal da Lusofonia Afro-Brasileira",
    "Universidade Federal da Paraíba",
    "Universidade Federal do Cariri",
    "Universidade Federal de Alagoas",
    "Universidade Federal de Campina Grande",
    "Universidade Federal de Pernambuco",
    "Universidade Federal de Sergipe",
    "Universidade Federal do Ceará",
    "Universidade Federal do Maranhão",
    "Universidade Federal do Oeste da Bahia",
    "Universidade Federal do Piauí",
    "Universidade Federal do Rio Grande do Norte",
    "Universidade Federal do Vale do São Francisco",
    "Universidade Federal Rural de Pernambuco",
    "Universidade Federal Rural do Semi-Árido",
    "Universidade Federal do Agreste de Pernambuco",
    "Universidade Federal do Delta do Parnaíba",

    # --- Institutos Federais ---
    "Instituto Federal de Educação, Ciência e Tecnologia de Alagoas",
    "Instituto Federal de Educação, Ciência e Tecnologia da Bahia",
    "Instituto Federal de Educação, Ciência e Tecnologia Baiano",
    "Instituto Federal de Educação, Ciência e Tecnologia do Ceará",
    "Instituto Federal de Educação, Ciência e Tecnologia do Maranhão",
    "Instituto Federal de Educação, Ciência e Tecnologia da Paraíba",
    "Instituto Federal de Educação, Ciência e Tecnologia de Pernambuco",
    "Instituto Federal de Educação, Ciência e Tecnologia do Sertão do Pernambuco",
    "Instituto Federal de Educação, Ciência e Tecnologia do Sertão do Piauí",
    "Instituto Federal de Educação, Ciência e Tecnologia do Sertão do Rio Grande do Norte",
    "Instituto Federal de Educação, Ciência e Tecnologia do Sertão do Rio Grande de Sergipe"
]

# transforma em regex única ignorando maiúsc/minúsc
instituicoes_regex = "|".join(instituicoes)


# ==============================
# Função para processar 1 arquivo XML
# ==============================
def processar_xml(xml_file):
    try:
        tree = ET.parse(xml_file)
        root = tree.getroot()

        # ====== Dados Gerais ======
        dados_gerais = root.find("DADOS-GERAIS")
        tabela_dados_gerais = pd.DataFrame([{
            "id_curriculo": root.attrib.get("NUMERO-IDENTIFICADOR"),
            "nome_completo": dados_gerais.attrib.get("NOME-COMPLETO"),
            "nome_citacao": dados_gerais.attrib.get("NOME-EM-CITACOES-BIBLIOGRAFICAS"),
            "nacionalidade": dados_gerais.attrib.get("PAIS-DE-NACIONALIDADE"),
            "cidade_nascimento": dados_gerais.attrib.get("CIDADE-NASCIMENTO"),
            "uf_nascimento": dados_gerais.attrib.get("UF-NASCIMENTO"),
            "pais_nascimento": dados_gerais.attrib.get("PAIS-DE-NASCIMENTO")
        }])

        # ====== Formação Acadêmica ======
        formacoes = []
        for f in dados_gerais.find("FORMACAO-ACADEMICA-TITULACAO"):
            formacoes.append({
                "id_curriculo": root.attrib.get("NUMERO-IDENTIFICADOR"),
                "tipo": f.tag,
                "instituicao": f.attrib.get("NOME-INSTITUICAO"),
                "curso": f.attrib.get("NOME-CURSO"),
                "ano_inicio": f.attrib.get("ANO-DE-INICIO"),
                "ano_conclusao": f.attrib.get("ANO-DE-CONCLUSAO"),
                "status": f.attrib.get("STATUS-DO-CURSO"),
                "titulo_trabalho": f.attrib.get("TITULO-DO-TRABALHO-DE-CONCLUSAO-DE-CURSO", "")
            })
        tabela_formacao = pd.DataFrame(formacoes)

        # ====== Atuação Profissional ======
        atuacoes = []
        for atuacao in root.findall(".//ATUACAO-PROFISSIONAL"):
            instituicao = atuacao.attrib.get("NOME-INSTITUICAO", "")
            cod_instituicao = atuacao.attrib.get("CODIGO-INSTITUICAO", "")
            for vinculo in atuacao.findall("VINCULOS"):
                atuacoes.append({
                    "id_curriculo": root.attrib.get("NUMERO-IDENTIFICADOR"),
                    "Instituição": instituicao,
                    "Código Instituição": cod_instituicao,
                    "Tipo Vínculo": vinculo.attrib.get("TIPO-DE-VINCULO", ""),
                    "Enquadramento": vinculo.attrib.get("ENQUADRAMENTO-FUNCIONAL", ""),
                    "Outro Enquadramento": vinculo.attrib.get("OUTRO-ENQUADRAMENTO-FUNCIONAL-INFORMADO", ""),
                    "Início": f"{vinculo.attrib.get('MES-INICIO','')}/{vinculo.attrib.get('ANO-INICIO','')}",
                    "Fim": f"{vinculo.attrib.get('MES-FIM','')}/{vinculo.attrib.get('ANO-FIM','')}",
                    "Carga Horária Semanal": vinculo.attrib.get("CARGA-HORARIA-SEMANAL", ""),
                    "Dedicacao Exclusiva": vinculo.attrib.get("FLAG-DEDICACAO-EXCLUSIVA", ""),
                    "Vínculo Empregatício": vinculo.attrib.get("FLAG-VINCULO-EMPREGATICIO", ""),
                    "Outras Informações": vinculo.attrib.get("OUTRAS-INFORMACOES", "")
                })

        tabela_atuacoes = pd.DataFrame(atuacoes)

        return tabela_dados_gerais, tabela_formacao, tabela_atuacoes

    except Exception as e:
        print(f"Erro ao processar {xml_file}: {e}")
        return pd.DataFrame(), pd.DataFrame(), pd.DataFrame()


# ==============================
# Caminho base do Google Drive
# ==============================
base_path = "/content/drive/MyDrive/xml_files"
arquivos = glob.glob(os.path.join(base_path, "**/*.xml"), recursive=True)

# Cria a pasta FILTRADO se não existir
filtrado_path = os.path.join(base_path, "FILTRADO")
os.makedirs(filtrado_path, exist_ok=True)

todos_dados_gerais = []
todas_formacoes = []
todas_atuacoes = []

for arquivo in arquivos:
    dados, formacao, atuacao = processar_xml(arquivo)

    # ---- filtro pelas instituições ----
    if not formacao.empty:
        filtro_formacao = formacao["instituicao"].str.contains(instituicoes_regex, case=False, na=False)
    else:
        filtro_formacao = pd.Series([False])

    if not atuacao.empty:
        filtro_atuacao = atuacao["Instituição"].str.contains(instituicoes_regex, case=False, na=False)
    else:
        filtro_atuacao = pd.Series([False])

    # ---- só guarda se tiver alguma instituição da lista ----
    if filtro_formacao.any() or filtro_atuacao.any():
        todos_dados_gerais.append(dados)
        todas_formacoes.append(formacao)
        todas_atuacoes.append(atuacao)

        # copia o xml para a pasta FILTRADO
        destino = os.path.join(filtrado_path, os.path.basename(arquivo))
        shutil.copy2(arquivo, destino)

# ==============================
# Consolidar em tabelas finais
# ==============================
tabela_dados_gerais_final = pd.concat(todos_dados_gerais, ignore_index=True)
tabela_formacao_final = pd.concat(todas_formacoes, ignore_index=True)
tabela_atuacoes_final = pd.concat(todas_atuacoes, ignore_index=True)

print("Currículos filtrados:", len(tabela_dados_gerais_final))

# ==============================
# Salvar em CSV no Drive
# ==============================
tabela_dados_gerais_final.to_csv(os.path.join(base_path, "dados_filtrados.csv"), index=False)
tabela_formacao_final.to_csv(os.path.join(base_path, "formacao_filtrada.csv"), index=False)
tabela_atuacoes_final.to_csv(os.path.join(base_path, "atuacoes_filtradas.csv"), index=False)
